Options can be written on futures as well, infact those are the most liquid types of options.
For pricing these options, we first compute the future price lattice - 
$F_0 = E_Q[S_n]$ 
ie. the expected stock price, where q is risk neutral probability.
Then the option prices are computed, on top of the future price lattice

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from math import exp, sqrt

In [14]:
def Parameter(T,n,sigma,r,c):
    """Parameter calculation"""  
    '''T is the time to maturity and n is the number of periods'''
    '''sigma is the volatility'''
    '''r is the the continuously compounded interest rate'''
    dt = T/n
    u = exp(sigma * sqrt(dt))
    d = 1/u
    
    q1 = (exp((r-c)*dt)-d)/(u-d)
    q2 = 1-q1
    R = exp(r*dt)
    
    return (u, d, R, q1, q2)

In [15]:
def bionomial_model(T,n,sigma,r,c, S):
    '''S is the stock price at t = 0'''
    '''u and d are the multiplier with which the stock price "S" moves up and down respectively'''
    '''R is risk free interest rate'''
    '''n is the number of time periods until maturity'''
    
    u, d, R, q1, q2 = Parameter(T,n,sigma,r,c)
    #Calculate stock trr, ie. stock prices at every time interval
    stockTree = np.zeros((n+1, n+1))  
    stockTree[0,0] = S
    for i in range(1,n+1):
        stockTree[0,i] = stockTree[0, i-1]*u
        for j in range(1,n+1):
            stockTree[j,i] = stockTree[j-1, i-1]*d
    return stockTree
    

In [16]:
def FuturesOptionEU(T,n,N,S,r,c,sigma,K,cp):
    """European Futures Option Pricing"""
    u, d, R, q1, q2 = Parameter(T,N,sigma,r,c)
    
    stockTree = bionomial_model(T,N,sigma,r,c, S)
    futuresTree = np.zeros((N+1, N+1))
    optionTree = np.zeros((n+1,n+1))           
            
    # compute the futures tree
    for j in range(N+1):
        futuresTree[j, N] = stockTree[j, N]
    
    for i in range(N-1, -1,-1): 
        for j in range(i+1):
            futuresTree[j,i] = q1 * futuresTree[j, i+1] + q2 * futuresTree[j+1, i+1]
    
    
    # compute the option tree
    for j in range(n+1):
        optionTree[j, n] = max(0, cp * (futuresTree[j, n]-K))
    
    for i in range(n-1,-1,-1):
        for j in range(i+1):
            optionTree[j, i] = (q1 * optionTree[j, i+1] + q2 * optionTree[j+1, i+1])/R                 
      
    print(optionTree)
    return optionTree[0,0]

In [19]:
FuturesOptionEU(3, 3, 3, 100, 1.02, 1.04, 0.2, 95, 0.4)

[[ 0.25254394  1.40749212  7.38908924 34.88475202]
 [ 0.          0.22697767  1.56974349 10.85611033]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]]


0.25254394480674297